# 概要
- BB1のscaffoldの情報を使って、FOLDを設定する
- bb1~3と同様にbb1もindex化して列を足し、scaffold用の辞書も準備

In [1]:
import os
from pathlib import Path

def is_kaggle_kernel():
    return os.path.exists('/kaggle/working')

if is_kaggle_kernel():

    BASE_DIR = Path("/kaggle")
    DATA_DIR = BASE_DIR / "input"
    OUTPUT_DIR = BASE_DIR / "working"
    print('on kaggle notebook')

else:
    BASE_DIR = Path(os.getcwd()) / './../'
    DATA_DIR = BASE_DIR / "data"
    OUTPUT_DIR = BASE_DIR / "output/eda"
    
class paths:    
    DATA_DIR = DATA_DIR
    TRAIN_PATH = DATA_DIR / "train.parquet"
    TEST_PATH = DATA_DIR / "test.parquet"
    OUTPUT_DIR = OUTPUT_DIR
    SHRUNKEN_DATA_DIR = DATA_DIR / "shrunken-train-set"
    
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [2]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import os
import gc
import math
import numpy as np
import pandas as pd
from glob import glob
# import duckdb
# import lightgbm as lgb
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from rdkit.Chem.Scaffolds import MurckoScaffold

import pickle

# 乱数をデフォルトで固定
# 乱数をデフォルトで固定
import random
np.random.seed(0)
random.seed(0)

In [4]:
bb_cols = ['buildingblock1_smiles', 'buildingblock2_smiles','buildingblock3_smiles', 
        #    'molecule_smiles'
           ]
TARGETS = ['binds_BRD4', 'binds_HSA','binds_sEH']

df_train = pd.read_parquet(paths.DATA_DIR / 'shrunken-train-set/train.parquet', columns=bb_cols + TARGETS)
df_test = pd.read_parquet(paths.DATA_DIR / 'shrunken-train-set/test.parquet', columns=bb_cols)

# building block smiles
# NOTE: trainとtestのindexとsmilesは一致していないっぽい
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_1.p', 'rb') as file:
    train_dicts_bb1 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_2.p', 'rb') as file:
    train_dicts_bb2 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_dict_reverse_3.p', 'rb') as file:
    train_dicts_bb3 = pickle.load(file)

with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_1_test.p', 'rb') as file:
    test_dicts_bb1 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_2_test.p', 'rb') as file:
    test_dicts_bb2 = pickle.load(file)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_dict_reverse_3_test.p', 'rb') as file:
    test_dicts_bb3= pickle.load(file)
    
train_dicts_bb1_reverse = {val:key for key, val in train_dicts_bb1.items()}
train_dicts_bb2_reverse = {val:key for key, val in train_dicts_bb2.items()}
train_dicts_bb3_reverse = {val:key for key, val in train_dicts_bb3.items()}
test_dicts_bb1_reverse = {val:key for key, val in test_dicts_bb1.items()}
test_dicts_bb2_reverse = {val:key for key, val in test_dicts_bb2.items()}
test_dicts_bb3_reverse = {val:key for key, val in test_dicts_bb3.items()}

In [5]:
bb1_smiles_train = [smiles for smiles in train_dicts_bb1.values()]
bb2_smiles_train = [smiles for smiles in train_dicts_bb2.values()]
bb3_smiles_train = [smiles for smiles in train_dicts_bb3.values()]
bb1_smiles_test = [smiles for smiles in test_dicts_bb1.values()]
bb2_smiles_test = [smiles for smiles in test_dicts_bb2.values()]
bb3_smiles_test = [smiles for smiles in test_dicts_bb3.values()]

bb1_mols_train = [Chem.MolFromSmiles(smiles) for smiles in bb1_smiles_train]
bb2_mols_train = [Chem.MolFromSmiles(smiles) for smiles in bb2_smiles_train]
bb3_mols_train = [Chem.MolFromSmiles(smiles) for smiles in bb3_smiles_train]
bb1_mols_test = [Chem.MolFromSmiles(smiles) for smiles in bb1_smiles_test]
bb2_mols_test = [Chem.MolFromSmiles(smiles) for smiles in bb2_smiles_test]
bb3_mols_test = [Chem.MolFromSmiles(smiles) for smiles in bb3_smiles_test]

## 分子をscaffoldごとにクラスタリングする

In [6]:
# bb1のscaffoldを取得
scaffold_dict = {}
for smiles in tqdm(bb1_smiles_train):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    
    scaffold_dict[smiles] = scaffold_smiles

100%|████████████████████████████████████████████████████████████████████████████████████| 271/271 [00:00<00:00, 3849.68it/s]


In [7]:
# ※ train dataの処理
# scaffold → intの辞書を作成
scaffold_list = list(set([s for s in scaffold_dict.values()]))
scaffold_list.sort()

scaffold_idx_dict = {s:num for num, s in enumerate(scaffold_list)}
scaffold_idx_dict_reverse = {num:s for num, s in enumerate(scaffold_list)}

# 保存
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_scaffold_dict_1.p', mode='wb') as f:
    pickle.dump(scaffold_idx_dict, f)
with open(paths.SHRUNKEN_DATA_DIR / 'train_dicts/BBs_scaffold_dict_reverse_1.p', mode='wb') as f:
    pickle.dump(scaffold_idx_dict_reverse, f)

In [8]:
# タニモト係数に応じてグループ分け
from rdkit.Chem import AllChem
from rdkit import DataStructs

scaffold_mols = [Chem.MolFromSmiles(s) for s in scaffold_list]
morgan_fp = [AllChem.GetMorganFingerprintAsBitVect(x, 2, 2048) for x in scaffold_mols]

dis_matrix = [DataStructs.BulkTanimotoSimilarity(morgan_fp[i], morgan_fp, returnDistance=True) for i in range(len(morgan_fp))]
### numpy.ndarrayへの変換
dis_array = np.array(dis_matrix)
dis_array.shape ### (5000, 5000)

(63, 63)

In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=5)
kmeans.fit(dis_array)

print('各クラスタのscaffold数')
print(pd.value_counts(kmeans.labels_))

fold_scaffold_dict = {i: [] for i in range(5)}
for n,j in enumerate(kmeans.labels_):
    fold_scaffold_dict[j].append(scaffold_list[n])
    
# fold1のscaffoldの4番目だけをfold0にして、それ以外ををfold0に押し込む（圧倒的に数が多いので）
# num = 4
# fold_scaffold_dict[0].extend(fold_scaffold_dict[1][:num] + fold_scaffold_dict[1][num+1:])
# fold_scaffold_dict[1] = [fold_scaffold_dict[1][num]]

print('各クラスタのscaffold数(修正後)')
for fold, s_list in fold_scaffold_dict.items():
    print(fold, len(s_list))

fold_scaffold_dict_reverse = {}
# key, valの入れ替え
for fold, s_list in fold_scaffold_dict.items():
    for s in s_list:
        fold_scaffold_dict_reverse[s] = fold

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


各クラスタのscaffold数
3    19
4    16
0    15
1    12
2     1
dtype: int64
各クラスタのscaffold数(修正後)
0 15
1 12
2 1
3 19
4 16


In [10]:
def bb1_idx_to_fold(bb1_idx):
    bb1_smiles = train_dicts_bb1[bb1_idx]
    bb1_scaffold_smi = scaffold_dict[bb1_smiles]
    fold = fold_scaffold_dict_reverse[bb1_scaffold_smi]
    return fold

# 辞書にする
bb1idx2fold = {bb1_idx:bb1_idx_to_fold(bb1_idx) for bb1_idx in df_train['buildingblock1_smiles'].unique()}

df_train['fold'] = df_train['buildingblock1_smiles'].map(bb1idx2fold)
df_train['fold'].value_counts()

3    44307006
0    27966077
4    10163121
1     8718245
2     7261161
Name: fold, dtype: int64

In [11]:
# >>> fold2のbinds_sEHだけ以上に多い
for fold in range(5):
    print(f'fold{fold}')
    print(df_train[df_train['fold'] == fold][TARGETS].mean())
    print('---')

fold0
binds_BRD4    0.005811
binds_HSA     0.004069
binds_sEH     0.004720
dtype: float64
---
fold1
binds_BRD4    0.002707
binds_HSA     0.003055
binds_sEH     0.001804
dtype: float64
---
fold2
binds_BRD4    0.012648
binds_HSA     0.004856
binds_sEH     0.003451
dtype: float64
---
fold3
binds_BRD4    0.002994
binds_HSA     0.004585
binds_sEH     0.002315
dtype: float64
---
fold4
binds_BRD4    0.004564
binds_HSA     0.002907
binds_sEH     0.044195
dtype: float64
---


## scaffold idx を df に足しておく

In [12]:
def bb1_idx_to_scaffold_idx(bb1_idx):
    bb1_smiles = train_dicts_bb1[bb1_idx]
    bb1_scaffold_smi = scaffold_dict[bb1_smiles]
    scaffold_idx = scaffold_idx_dict[bb1_scaffold_smi]
    return scaffold_idx

# 辞書にする（bb1_idx → scaffold_idx）
bb1idx2scaidx = {bb1_idx:bb1_idx_to_scaffold_idx(bb1_idx) for bb1_idx in df_train['buildingblock1_smiles'].unique()}

df_train['bb1_scaffold_idx'] = df_train['buildingblock1_smiles'].map(bb1idx2scaidx)

### test data

In [13]:
# bb1のscaffoldを取得
scaffold_dict = {}
for smiles in tqdm(bb1_smiles_test):
    mol = Chem.MolFromSmiles(smiles)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_smiles = Chem.MolToSmiles(scaffold)
    
    scaffold_dict[smiles] = scaffold_smiles

100%|████████████████████████████████████████████████████████████████████████████████████| 341/341 [00:00<00:00, 3763.52it/s]


In [14]:
# scaffold → intの辞書を作成
scaffold_list = list(set([s for s in scaffold_dict.values()]))
scaffold_list.sort()

scaffold_idx_dict = {s:num for num, s in enumerate(scaffold_list)}
scaffold_idx_dict_reverse = {num:s for num, s in enumerate(scaffold_list)}

# 保存
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_scaffold_dict_1.p', mode='wb') as f:
    pickle.dump(scaffold_idx_dict, f)
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_scaffold_dict_reverse_1.p', mode='wb') as f:
    pickle.dump(scaffold_idx_dict_reverse, f)

In [15]:
def bb1_idx_to_scaffold_idx(bb1_idx):
    bb1_smiles = test_dicts_bb1[bb1_idx]
    bb1_scaffold_smi = scaffold_dict[bb1_smiles]
    scaffold_idx = scaffold_idx_dict[bb1_scaffold_smi]
    return scaffold_idx

# 辞書にする（bb1_idx → scaffold_idx）
bb1idx2scaidx = {bb1_idx:bb1_idx_to_scaffold_idx(bb1_idx) for bb1_idx in df_test['buildingblock1_smiles'].unique()}
df_test['bb1_scaffold_idx'] = df_test['buildingblock1_smiles'].map(bb1idx2scaidx)

# コード実行時に必要になるので辞書も保存しておく
with open(paths.SHRUNKEN_DATA_DIR / 'test_dicts/BBs_idx_to_scaffold_idx_dict_1.p', mode='wb') as f:
    pickle.dump(bb1idx2scaidx, f)

In [16]:
## 保存
df_train.to_parquet(paths.SHRUNKEN_DATA_DIR / 'train_fold.parquet')
df_test.to_parquet(paths.SHRUNKEN_DATA_DIR / 'test_fold.parquet')